## Exploring the Dataset

### Importing all the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

### Data Loading

In [ ]:
reviews = pd.read_json('data\Yelp-JSON\Yelp JSON\Extracted\yelp_academic_dataset_review.json', lines=True, nrows=500000)
print(reviews.shape)
reviews.head()